<a href="https://colab.research.google.com/github/ayoola-babatunde/10-days-of-code-1/blob/master/2019_Flattening_Data_files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a script to convert the 2019 data (Tattum only) to proper cleanup format. This is in a separate colab file because it is too different from 2020-22. 

-Ayo

In [1]:
import pandas as pd
import numpy as np

### 2019

In [2]:
xls19 = pd.ExcelFile('https://docs.google.com/spreadsheets/d/e/2PACX-1vRzmUxWr-YprmvSuA6uS2kNnochdPj4dLviy-09TGNhtfhOzEYdobQTo4RRye1pIgM2gDBFJgAjcwdG/pub?output=xlsx')

all_dfs19 = pd.read_excel(xls19, sheet_name = None, header=None)
ppl = list(all_dfs19.keys())

In [3]:
h_row = ['End of Each Week', 'Date', 'Session #', 'Words per Min Cold (1 Minute Reading) ', '# of Errors', 'Fluency', 'Book Reading Level', 'Concept', 'Book Title (Page #)']

good_sheets = [] #can be automated
bad_sheets = [] #to be done manually

for sheet in ppl: 
    if all_dfs19[sheet].iloc[8,:9].to_list() == h_row: 
        good_sheets.append(sheet)
    else: 
        bad_sheets.append(sheet)

In [4]:
def fn_2019(good_sheets, all_dfs, year): 
    general = ['Student Full Name', 'Age', 'Grade', 'Tutor Full Name', 'Start', 
               'Program Partner Org/School', 'Program Enrollment', 'Year']
    tattum = ['Student Full Name', 'Date', 'Session #', 'Concept', 'Tattum Book, page #', 
                'Grade Level', 'WPM', 'Errors', 'Fluency', 'Tutor Full Name', 'Year']

    generaldf = pd.DataFrame(columns=general)
    tattumdf = pd.DataFrame(columns=tattum)
    for sheet in good_sheets: 
        currentdf = all_dfs[sheet]

        #generaldf stuff
        student_name = f"{currentdf.iloc[1,1]} {currentdf.iloc[2,1]}"
        age = currentdf.iloc[3,1]
        grade = currentdf.iloc[4,1]
        tutor_name = currentdf.iloc[5,1]
        start = currentdf.iloc[6,1]
        year = year
        current_gen_info = [student_name, age, grade, tutor_name, start, "", "", year]

        generaldf = generaldf.append(pd.DataFrame(columns = general, data = [current_gen_info]))

        #tattum stuff

        #get index of last row of tattum data aka row that says "Min"
        current_tattum_end = currentdf.iloc[:, 1][currentdf.iloc[:, 1] == "Min"].index.tolist()[0]

        current_tattum_df = currentdf.iloc[8:current_tattum_end, :]
        current_tattum_df.columns = current_tattum_df.iloc[0]
        current_tattum_df = current_tattum_df[1:]
        current_tattum_df.loc[:, 'Student Full Name'] = student_name
        current_tattum_df.loc[:, 'Tutor Full Name'] = tutor_name
        current_tattum_df.loc[:, 'Year'] = year
        current_tattum_df.rename(columns={"Book Title (Page #)": 'Tattum Book, page #', 
                                        "Book Reading Level": 'Grade Level', 
                                        "Words per Min Cold (1 Minute Reading) ": 'WPM', 
                                        "# of Errors": 'Errors'}, inplace=True)
        current_tattum_df_re = current_tattum_df[tattum]
        tattumdf = tattumdf.append(current_tattum_df_re)

    return generaldf, tattumdf.dropna(thresh=8)
 


In [5]:
generaldf19, tattumdf19 = fn_2019(good_sheets, all_dfs19, 2019)

In [6]:
generaldf19.to_csv('2019gendf.csv')
tattumdf19.to_csv('2019tatdf.csv')

### 2018

In [7]:
#link goes to copy of 2018 sheet
xls18 = pd.ExcelFile('https://docs.google.com/spreadsheets/d/e/2PACX-1vSEmY0rAsgLDhFR7ASGpGHggTBuaG5MNQqPtbLg8jmo-yt7kX_85s0ARw0JShLE9fO8QJ5TmOrSxT2s/pub?output=xlsx')

all_dfs18 = pd.read_excel(xls18, sheet_name = None, header=None)
ppl = list(all_dfs18.keys())

/usr/local/lib/python3.7/dist-packages/openpyxl/worksheet/_reader.py:211: UserWarning: Cell B22 is marked as a date but the serial value 6673664.0 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)


In [8]:
h_row = ['End of Each Week', 'Date', 'Session #', 'Words per Min Cold (1 Minute Reading) ', '# of Errors', 'Fluency', 'Book Reading Level', 'Concept', 'Book Title (Page #)']

good_sheets_9 = [] #data starts row 9, can be automated like 2019
good_sheets_7 = [] #data starts row 7, can be automated slightly differently
bad_sheets = [] #to be done manually

for sheet in ppl: 
    if all_dfs18[sheet].iloc[6,:9].to_list() == h_row: 
        good_sheets_7.append(sheet)
    elif all_dfs18[sheet].iloc[8,:9].to_list() == h_row: 
        good_sheets_9.append(sheet)
    else: 
        bad_sheets.append(sheet)

In [9]:
generaldf18_1, tattumdf18_1 = fn_2019(good_sheets_9, all_dfs18, 2018)

In [10]:
#add two blank rows to make format same as row 9 starts
for sheet in good_sheets_7: 
   currentdf = all_dfs18[sheet]
   blankdf = pd.DataFrame([[""] * len(currentdf.columns)]*2, columns = currentdf.columns)
   currentdf = blankdf.append(currentdf, ignore_index=True)
   all_dfs18[sheet] = currentdf

In [11]:
generaldf18_2, tattumdf18_2 = fn_2019(good_sheets_7, all_dfs18, 2018)

In [12]:
generaldf18_1.append(generaldf18_2).to_csv('2018gendf.csv')
tattumdf18_1.append(tattumdf18_2).to_csv('2018tatdf.csv')